In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
import IPython
from openai import OpenAI

In [4]:
client = OpenAI()

#

### 1. Prepare Batch Requests File

In [5]:
!ls -l static

total 3488
-rw-rw-r-- 1 lal lal     506 Aug 13 14:08 batch-requests.jsonl
-rw-rw-r-- 1 lal lal 1368003 Jul 24 16:04 cat-mask.png
-rw-rw-r-- 1 lal lal 1379936 Jul 24 16:04 cat.png
-rw-rw-r-- 1 lal lal    7342 Jul 26 19:36 code-craft-marketplace.jsonl
-rw-rw-r-- 1 lal lal   23747 Jul 29 14:18 different-format.aac
-rw-rw-r-- 1 lal lal   58080 Aug  4 18:31 echo.mp3
-rw-rw-r-- 1 lal lal   79680 Jul 31 23:27 german.mp3
-rw-rw-r-- 1 lal lal  169265 Aug  4 18:17 image.jpg
-rw-rw-r-- 1 lal lal   58560 Aug  4 18:31 nova.mp3
-rw-rw-r-- 1 lal lal   56160 Jul 24 16:04 onyx.mp3
-rw-rw-r-- 1 lal lal   20723 Aug  4 18:31 output.aac
-rw-rw-r-- 1 lal lal   86880 Jul 20 18:03 real-time.mp3
-rw-rw-r-- 1 lal lal   61440 Aug  4 18:31 spanish.mp3
-rw-rw-r-- 1 lal lal   57120 Aug  4 18:31 speech.mp3
-rw-rw-r-- 1 lal lal   78720 Jul 24 16:04 speech-with-speed.mp3
-rw-rw-r-- 1 lal lal   28800 Aug  4 18:31 speed.mp3
-rw-rw-r-- 1 lal lal    3640 Jul 26 16:40 training-data.jsonl


In [8]:
!cat static/batch-requests.jsonl

{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}
{"custom_id": "request-2", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo", "messages": [{"role": "system", "content": "You are an unhelpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}

#

### 2. Upload Batch Requests File

In [10]:
batch_file = client.files.create(
  file=open("static/batch-requests.jsonl", "rb"),
  purpose="batch"
)

In [11]:
batch_file_id = batch_file.id
batch_file_id

'file-kmp5MJotsczgCij48YKPd0kY'

#

### 3. Creating the Batch Job

In [12]:
batch_job = client.batches.create(
    input_file_id=batch_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h", # For now, the completion window can only be set to 24h
)

In [14]:
batch_job_id = batch_job.id
batch_job_id

'batch_hR8iQ3fmdaVnm2yl9Ue4hyyn'

#

### 4. Checking the Status of a Batch

In [19]:
batch_job = client.batches.retrieve(batch_job_id)
batch_job

Batch(id='batch_hR8iQ3fmdaVnm2yl9Ue4hyyn', completion_window='24h', created_at=1723540396, endpoint='/v1/chat/completions', input_file_id='file-kmp5MJotsczgCij48YKPd0kY', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1723540436, error_file_id=None, errors=None, expired_at=None, expires_at=1723626796, failed_at=None, finalizing_at=1723540435, in_progress_at=1723540397, metadata=None, output_file_id='file-d2B4WxkJFCIoaJ3FX4zBcqMP', request_counts=BatchRequestCounts(completed=2, failed=0, total=2))

In [20]:
output_file_id = batch_job.output_file_id
output_file_id

'file-d2B4WxkJFCIoaJ3FX4zBcqMP'

#

### 5. Retrieving the Results

In [49]:
batch_response = client.files.content(output_file_id)
print(batch_response.text)

{"id": "batch_req_peAo2LcKlQ1salX6MbVcQcxa", "custom_id": "request-1", "response": {"status_code": 200, "request_id": "3d2f011eba4c94aaca703a3ad0f39930", "body": {"id": "chatcmpl-9vht26GnvBgkXJqiCdq6LN0LF4Cgx", "object": "chat.completion", "created": 1723540424, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Hello! How can I assist you today?", "refusal": null}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 20, "completion_tokens": 9, "total_tokens": 29}, "system_fingerprint": null}}, "error": null}
{"id": "batch_req_sVDq0EQbpwWLubg3FlusqP3R", "custom_id": "request-2", "response": {"status_code": 200, "request_id": "5f9aa6a03f1f7bcc63cc0f4fbf823b2f", "body": {"id": "chatcmpl-9vht9rvY7Ep3apAhwkwidSgyXfT0h", "object": "chat.completion", "created": 1723540431, "model": "gpt-3.5-turbo-0125", "choices": [{"index": 0, "message": {"role": "assistant", "content": "Greetings. How may I not assist you today?", "refus

In [51]:
import json
for request in batch_response.text.strip().split('\n'):
    request = json.loads(request)
    print(request['response']['body']['choices'][0]['message']['content'], '\n\n')

Hello! How can I assist you today? 


Greetings. How may I not assist you today? 




The output .jsonl file will have one response line for every successful request line in the input file. Any failed requests in the batch will have their error information written to an error file that can be found via the batch's error_file_id

### Batch Expiration